In [1]:
from ortools.linear_solver import pywraplp
from instances import *
from pathlib import Path
import numpy as np
from itertools import chain

In [2]:
shapes = [
  (100,100),
  (150,100),
  (100,150)
]

In [3]:
create_instances(shapes=shapes, random_state=42)

In [4]:
output_folder = Path() / 'instances'

## Modelo Original

In [5]:
for idx,details in enumerate(get_instances()):
  s,d,c = details
  ns = len(s)
  nd = len(d)
  x = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns):
    x.append([])  
    for j in range(nd):
      x[i].append(solver.NumVar(0, solver.Infinity(), f'x_{i}_{j}'))

  for i in range(ns):
    solver.Add(solver.Sum([x[i][j] for j in range(nd)]) <= s[i])

  for j in range(nd):
    solver.Add(solver.Sum([(x[i][j]) for i in range(ns) ]) >= d[j])

  for i in range(ns):
    for j in range(nd):
      objective_terms.append(c[i][j] * x[i][j])

  solver.Minimize(solver.Sum(objective_terms))
  status = solver.Solve()

  # with (output_folder / f'{idx}_model.txt').open('w') as model_file:
  #   model_file.write(solver.ExportModelAsLpFormat(False))
  #   model_file.close()

  with (output_folder / f'{idx}_sol.txt').open('w') as output_file:
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
      output_file.write(f'Total = { np.round(solver.Objective().Value(), 2) }\n')

      # for i in range(ns):
      #   for j in range(nd):
      #     if x[i][j].solution_value() > 0.0:
      #         output_file.write(
      #               f'S_{i} envia { np.round(x[i][j].solution_value(), 2) } para D_{j} ' +
      #               f'ao custo de { np.round(c[i][j] * x[i][j].solution_value(), 2) }\n')
    
    else:
      output_file.write('No solution found\n')
    

## Usando integralidade

In [6]:
for idx,details in enumerate(get_instances()):
  s,d,c = details
  ns = len(s)
  nd = len(d)
  x = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns):
    x.append([])
    for j in range(nd):
      x[i].append(solver.IntVar(0, solver.Infinity(), f'x_{i}_{j}'))

  for i in range(ns):
    solver.Add(solver.Sum([x[i][j] for j in range(nd)]) <= s[i])

  for j in range(nd):
    solver.Add(solver.Sum([(x[i][j]) for i in range(ns) ]) >= d[j])

  for i in range(ns):
    for j in range(nd):
      objective_terms.append(c[i][j] * x[i][j])

  solver.Minimize(solver.Sum(objective_terms))
  status = solver.Solve()

  # with (output_folder / f'{idx}_model_int.txt').open('w') as model_file:
  #   model_file.write(solver.ExportModelAsLpFormat(False))
  #   model_file.close()

  with (output_folder / f'{idx}_sol_int.txt').open('w') as output_file:
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
      output_file.write(f'Total = { np.round(solver.Objective().Value(), 2) }\n')

      # for i in range(ns):
      #   for j in range(nd):
      #     if x[i][j].solution_value() > 0.0:
      #         output_file.write(
      #               f'S_{i} envia { x[i][j].solution_value() } para D_{j} ' +
      #               f'ao custo de { np.round(c[i][j] * x[i][j].solution_value(), 2) }\n')
    
    else:
      output_file.write('No solution found\n')

## Solução do dual

In [8]:
for idx,details in enumerate(get_instances()):
  s,d,c = details
  ns = len(s)
  nd = len(d)
  y = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns + nd):
    y.append(solver.NumVar(0.0, solver.Infinity(), f'y_{i}'))

  for i in range(ns):
    for j in range(nd):
      solver.Add( y[i] - y[j + ns] >= -c[i][j] )

  for i, b in zip(list(range(ns+nd)), chain(s, list(map(lambda x: -x, d)))):
    objective_terms.append(y[i] * b)

  solver.Minimize(solver.Sum(objective_terms))

  status = solver.Solve()

  # with (output_folder / f'{idx}_model_dual.txt').open('w') as model_file:
  #   model_file.write(solver.ExportModelAsLpFormat(False))
  #   model_file.close()

  with (output_folder / f'{idx}_sol_dual.txt').open('w') as output_file:
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
      output_file.write(f'Total = { np.round(solver.Objective().Value() * -1, 2) }\n')
    else:
      output_file.write('No solution found\n')